In [19]:
import pandas as pd
import keras
import numpy as np
import matplotlib
import sklearn

In [2]:
location='../data/'

dipolesFile= location + 'dipole_moments.csv'
magneticShieldingFile= location + 'magnetic_shielding_tensors.csv'
mullikenFile= location + 'mulliken_charges.csv'
potentialEnergyFile= location + 'potential_energy.csv'
sampleFile= location + 'sample_submission.csv'
scalarCouplingFile= location + 'scalar_coupling_contributions.csv'
structuresFile= location + 'structures.csv'
testFile= location + 'test.csv'
trainFile= location + 'train.csv'
#File= location + '.csv'
#File= location + '.csv'

In [3]:
dipoles=pd.read_csv(dipolesFile)
magneticShielding=pd.read_csv(magneticShieldingFile)
mulliken=pd.read_csv(mullikenFile)
potentialEnergy=pd.read_csv(potentialEnergyFile)
sample=pd.read_csv(sampleFile)
scalarCoupling=pd.read_csv(scalarCouplingFile)
structures=pd.read_csv(structuresFile)
test=pd.read_csv(testFile)
train=pd.read_csv(trainFile)

In [4]:
#test.info #2505542
#train.info #4658147

# Feature Engineering

### Dipoles

In [5]:
dipoles['polar']=((dipoles['X']==0) & (dipoles['Y']==0) & (dipoles['Z']==0))

In [6]:
dipoles.describe()

,X,Y,Z
count,85003.000000,85003.000000,85003.000000
mean,-0.019937,0.093764,0.252982
std,2.283738,1.739395,1.027573
min,-22.958000,-9.252300,-6.028500
25%,-1.363250,-0.941950,-0.247750
50%,-0.000100,0.081400,0.117600
75%,1.311000,1.166750,0.902050
max,21.873800,13.005600,6.880100


In [7]:
dipoles.head()

,molecule_name,X,Y,Z,polar
0,dsgdb9nsd_000001,0.0000,0.0,0.0000,True
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256,False
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511,False
3,dsgdb9nsd_000005,0.0000,0.0,-2.8937,False
4,dsgdb9nsd_000007,0.0000,0.0,0.0000,True


### Magnetic Shielding

In [8]:
magneticShielding.head()

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
0,dsgdb9nsd_000001,0,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
1,dsgdb9nsd_000001,1,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861
2,dsgdb9nsd_000001,2,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967
3,dsgdb9nsd_000001,3,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579
4,dsgdb9nsd_000001,4,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013


In [9]:
magneticShieldingUnified=magneticShielding.groupby('molecule_name').sum()
magneticShieldingUnified['count']=magneticShielding.groupby('molecule_name')['atom_index'].count()

In [10]:
magneticShieldingUnified=magneticShieldingUnified.reset_index()

In [11]:
magneticShieldingUnified.head()

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ,count
0,dsgdb9nsd_000001,10,321.1575,0.0000,-0.0001,-8.881784e-16,321.1596,0.0007,-0.0002,0.0009,321.1590,5
1,dsgdb9nsd_000002,6,378.6791,0.0002,0.0127,2.000000e-04,378.6755,-0.0002,0.0167,-0.0003,321.4628,4
2,dsgdb9nsd_000003,3,359.4450,0.0000,0.0000,0.000000e+00,434.1190,0.0000,0.0000,0.0000,387.4142,3
3,dsgdb9nsd_000005,3,-155.7703,0.0000,0.0000,0.000000e+00,-155.7703,0.0000,0.0000,0.0000,656.9822,3
4,dsgdb9nsd_000007,28,571.0991,0.0008,0.0017,3.000000e-04,535.8459,0.0014,0.0016,0.0013,535.8345,8


### Mulliken

In [12]:
mullikenUnified=mulliken.groupby('molecule_name').sum().reset_index()

In [13]:
mullikenUnified

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,10,0.000000e+00
1,dsgdb9nsd_000002,6,1.000000e-06
2,dsgdb9nsd_000003,3,0.000000e+00
3,dsgdb9nsd_000005,3,0.000000e+00
4,dsgdb9nsd_000007,28,1.249001e-16
5,dsgdb9nsd_000008,15,1.000000e-06
6,dsgdb9nsd_000009,21,1.000000e-06
7,dsgdb9nsd_000010,15,2.775558e-17
8,dsgdb9nsd_000011,21,1.000000e-06
9,dsgdb9nsd_000012,15,-5.551115e-17


In [14]:
sample.head()

,id,scalar_coupling_constant
0,4658147,0
1,4658148,0
2,4658149,0
3,4658150,0
4,4658151,0


In [15]:
train

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807600
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254800
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807400
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.254100
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.254800
7,7,dsgdb9nsd_000001,3,0,1JHC,84.809300
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.254300
9,9,dsgdb9nsd_000001,4,0,1JHC,84.809500


In [16]:
scalarCoupling.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [17]:
a=magneticShielding.merge(mulliken)
a=a.merge(dipoles)
a=a.merge(potentialEnergy)

In [18]:
a.head()

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ,mulliken_charge,X,Y,Z,polar,potential_energy
0,dsgdb9nsd_000001,0,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,-0.535689,0.0,0.0,0.0,True,-40.52368
1,dsgdb9nsd_000001,1,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861,0.133921,0.0,0.0,0.0,True,-40.52368
2,dsgdb9nsd_000001,2,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967,0.133922,0.0,0.0,0.0,True,-40.52368
3,dsgdb9nsd_000001,3,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579,0.133923,0.0,0.0,0.0,True,-40.52368
4,dsgdb9nsd_000001,4,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013,0.133923,0.0,0.0,0.0,True,-40.52368


# Predictions

In [22]:
train['type'].unique()

array(['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN'],
      dtype=object)

In [ ]:
X=a
y=

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
X, y = make_regression(n_features=4, n_informative=2,
                       random_state=0, shuffle=False)
regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X, y)  


regr.score(X, y)  